# Forecast verification
This is the main notebook of the study containing all the skill assessments.

* **Most use seasonal averages for verification. How to best aggregate data? Needed in daily resolution for hindcast. Initial timestep of model = 20 mins (correct?)... if I aggregate to seasonal averages I could have just as well downloaded the monthly aggregated data firsthand and not bothered with the large daily dataset... how can daily data be considered in the analysis?**
* see "Effective Sample Size" -> should I use it? see https://doi.org/10.1175/1520-0442(1999)012<1990:TENOSD>2.0.CO;2  Also compute the p values! Understand how this is used in forecast verification!

* Check more verification datasets in one HindcastEnsemble? [Note](https://youtu.be/EcMxImmMBec?t=8526) more than one verification dataset can be carried along! This allows to compare the skill for multiple observational datasets.

# Misc TODO

## Climpred things
* Make sure that your init and lead dimensions align.* --> maybe I need to have the same number of init dates as lead timesteps. Try the xarray align method??? See https://climpred.readthedocs.io/en/stable/quick-start.html
* Check: [Note](https://youtu.be/EcMxImmMBec?t=8647) a skill dimension is appended to the HindcastEnsemble object which allows to compare the skills of the forecast product and the skill of the reference forecast (e.g. persistence) -> good plot of the spatial distribution of skill in two rows, one for the forecast product and the other one for the reference product
* --> just not sure how e.g. the ACC is computed then if we are not even in anomaly space. Does climpred actually compute anomalies?

## EOF analysis
* compute the EOFs
    * https://ajdawson.github.io/eofs/latest/
    * read Wilks chapter

## Detrending
* https://climpred.readthedocs.io/en/stable/api/climpred.stats.rm_poly.html#climpred.stats.rm_poly
    * or better in XCLIM or straight away in Xarray?
    * see here: https://youtu.be/SKXUBD6DGao?t=563
* I will probably not need to worry about detrending at seasonal timescales... where is literature which specifies at which time intervals detrending becomes relevant? Do I need to justify?


In [5]:
# %%time
# # set up a local dask cluster for parallelisation and accessing the dashboard
# from dask.distributed import Client, LocalCluster
# cluster = LocalCluster()  # Launches a scheduler and workers locally
# client = Client(cluster)  # Connect to distributed cluster and override default
# # client = Client(address="127.0.0.1:8787")
# client

In [18]:


# Testing only:
# hc = hc.assign_coords({"month" : hc.init.dt.month.values}) # for testing to
# filter for inits on a specific month to compare results against https://meteoswiss-climate.shinyapps.io/skill_metrics/


C:\Users\sb123\.conda\envs\Thesis2\lib\site-packages\climpred\classes.py:987: UserWarning: HindcastEnsemble is chunked along dimensions ['init'] with more than one chunk. `HindcastEnsemble.chunks=Frozen({'init': (1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1), 'member': (25,), '

TypeError: 'function' object is not iterable

In [ ]:
convert_init_lead_to_valid_time_lead(hindcast.get_initialized())

In [ ]:
# Doing some visualisations and plausibility checks on our climpred
# HindcastEnsemble

# fig, ax = plt.subplots()
data = hindcast.get_observations().tp.values.flat
# import numpy as np
data[np.where(data < 0)[0]]

# sns.histplot(data=hindcast.get_observations().tp.values[:,0,0])

In [ ]:
%time
#
verification_e2o = hindcast.verify(
    metric="pearson_r", comparison="e2o", dim="init", alignment="same_inits"
)

In [ ]:
verification_e2o

**Check, do I still have a member dimension when verifying with m2o?**

**ToDo:** Also calculate the p - values and research the underlying
methodology (see Wilks 2019)

Da läuft noch was schief mit den Verification Alignments

In [ ]:
fig, ax = plt.subplots(
    ncols=2, figsize=(10, 5), subplot_kw={"projection": ccrs.PlateCarree()}
)
verification.sel({"lead": 1})["tp"].plot(ax=ax[0], vmin=0, vmax=0.6)
verification.sel({"lead": 100})["tp"].plot(ax=ax[1], vmin=0, vmax=0.6)

for ax in ax.flatten():
    ax.add_feature(cfeature.BORDERS)
    ax.add_feature(cfeature.COASTLINE)
plt.tight_layout()

In [ ]:
verification.dims

In [ ]:
fig, ax = plt.subplots()
y = verification.tp.sel({"longitude": 5, "latitude": 55}).values
x = verification.lead.values
ax.plot(x, y)
fig.show()

# TODO compute manually and compare the graphs!

In [ ]:
verification.tp.shape

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()})
verification.tp.sel({"lead": 30}).plot(ax=ax)
ax.add_feature(cfeature.COASTLINE)
fig.show()

In [ ]:
verification.tp.sel({"lead": 30})